In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
import tkinter as tk
import matplotlib
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
import threading

matplotlib.use("TkAgg")


class Animator:
    def __init__(self, update_freq: int = 100) -> None:

        # Create the main window
        self.plot_window = tk.Tk()

        # Create a figure and axis
        self.fig = Figure(figsize=(6, 4), dpi=100)
        self.ax = self.fig.add_subplot(111, projection="3d")

        # Create a canvas to display the plot
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.plot_window)
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

        # Initialize an empty list to store the data
        self.data = []

        self.plot_window.bind("<<PlotUpdate>>", self.update_plot)

        self.update_freq = update_freq
        self.call_num = 0

    # Define the callback function
    def callback_func(self, point, value):
        self.data.append((point, value))

        if self.call_num % self.update_freq == 0:
            self.plot_window.event_generate("<<PlotUpdate>>")  # Trigger the plot update event
            self.plot_window.update_idletasks()

        self.call_num = (self.call_num + 1) % self.update_freq

    # Define the function to update the plot
    def update_plot(self, event=None):
        points = np.array([point[:3] for point, value in self.data])
        values = np.array([value for point, value in self.data])
        self.ax.clear()
        if self.data:
            self.ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=values, cmap="viridis")
        self.ax.set_xlabel("X")
        self.ax.set_ylabel("Y")
        self.ax.set_zlabel("Z")
        self.canvas.draw()

In [ ]:
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectConfig
from loss_funcs import *
from algs import *

from eval_funcs import *
from evaluator import Evaluator

animator = Animator()

# Create a camera configuration
cam_config = CameraConfig(position=(0, 0, 0.1), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/world_mug.xml", cam_config, simulation_time=5)
sim_viewer = ViewSampler("data/world_mug_sim.xml", cam_config)

loss_func = MSE()

alg1 = ParticleSwarm(sim_viewer, loss_func=loss_func)
alg1_config = ParticleSwarm.Config(time_limit=50, population=30, num_iters=300)

alg2 = SimulatedAnnealing(sim_viewer, loss_func=loss_func)
alg2_config = SimulatedAnnealing.Config(time_limit=50, num_iters=100, samples_per_temp=150)

alg3 = RandomSampling(sim_viewer, loss_func=loss_func)
alg3_config = RandomSampling.Config(time_limit=10, num_samples=10000)

alg4 = DifferentialEvolution(sim_viewer, loss_func=loss_func)
alg4_config = DifferentialEvolution.Config(time_limit=10, population=30, mut_prob=0.3)

alg5 = UniformSampling(sim_viewer, loss_func=loss_func)
alg5.register_loss_callback(animator.callback_func)
alg5_config = UniformSampling.Config(time_limit=100, min_samples=10000, randomized=False)

evaluator = Evaluator(world_viewer, eval_func=IOU_Diff(method="mae"))

init_position = (0, 1.3, 0.3)
random_orientations = np.random.uniform(0, 2 * np.pi, size=(5, 3))
init_positions = [ObjectConfig(orient, init_position) for orient in random_orientations]

for alg, config in [(alg5, alg5_config)]:
    losses = evaluator.evaluate(alg, config, init_positions)
    print(f"{type(alg).__name__}: {losses}")

In [ ]:
cv.destroyAllWindows()